In [ ]:
#| default_exp feature_extraction

# Feature extraction

> Extracting meaningful features from the data is always important, but plays a key role when our data is of very high dimensionality. 

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [ ]:
#| hide
from nbdev.showdoc import *
import matplotlib.pyplot as plt

In [ ]:
#| hide
path_data = "/media/2tbraid/antonia/PSG/"

In [ ]:
#| exports
from multiprocessing.spawn import import_main_path
import os
from glob import glob
from collections import Counter
from typing import List, Dict

from rich.progress import track
import numpy as np
import pandas as pd
import mne
import yasa

from sleepstagingidal.data import *
from sleepstagingidal.dataa import *

## Amplitude-independent features

In our case, we have downsampled our data and have 3000 data points per epoch (30s fragment of the recording), but this is a huge amount of features to be fed into a model. Getting even further, we have the problem of calibration and normalization, which is aggravated even more when working with medical data (each hospital may work slightly different). Because of this, we are going to employ features that are independent of the amplitude of the signal: we are going to utilize mainly frequency-related features.

We can use the library `yasa` to perform a basic feature extraction:

In [ ]:
path_files = glob(os.path.join(path_data, "*.edf"))

In [ ]:
channels = ["C3", "C4", "A1", "A2", "O1", "O2", "LOC", "ROC", "LAT1", "LAT2", "ECGL", "ECGR", "CHIN1", "CHIN2"]

Before extracting the features, we are going to perform downsampling and a bandpass filter to keep only the low frequencies:

In [ ]:
raw = mne.io.read_raw_edf(path_files[0], preload=False, verbose=False)
# Downsample the data to 100 Hz
raw.resample(100)
# Apply a bandpass filter from 0.3 to 49 Hz
raw.filter(0.3, 49)

Once it's done, we can use the function `yasa.bandpower()` to extract frequency-related features from the data. Keep in mind that this function expects the data to be fed in epochs form, so we have to transform it first:

In [ ]:
epochs, sr = get_epochs(raw, channels=channels, verbose=False)
epochs

Using data from preloaded Raw for 719 events and 3000 original time points ...
1 bad epochs dropped


Number of events,718
Events,Sleep stage N1: 29Sleep stage N2: 323Sleep stage W: 366
Time range,0.000 – 29.990 sec
Baseline,off


In [ ]:
#| export

def calculate_bandpower(epochs, # Epochs object or 3D array [Epochs, Channels, Data].
                        sf=100, # Sampling frequency of the data.
                        ) -> np.array: # Numpy array of shape [Epochs, 6*Channels] representing 6 bands.
    """Extracts the bandpower per epoch and returns it as an array ready to be fed into a model."""
    n_channels = len(epochs.ch_names) if isinstance(epochs, mne.epochs.Epochs) else epochs.shape[1]
    bandpowers = np.empty(shape=(len(epochs), 6*n_channels))
    for i, epoch in enumerate(epochs):
        df_bandpower = yasa.bandpower(epoch, sf=sf)
        df_bandpower.drop(['TotalAbsPow', 'Relative', 'FreqRes'], axis=1, inplace=True)
        df_bandpower = df_bandpower.to_numpy().flatten()
        if len(df_bandpower) != bandpowers.shape[-1]: raise ValueError("Shape mismatch between calculated features and pre-allocated array.")
        bandpowers[i] = df_bandpower
    return bandpowers

In [ ]:
%%time
bandpowers = calculate_bandpower(epochs, sf=sr)

CPU times: user 4.34 s, sys: 0 ns, total: 4.34 s
Wall time: 4.34 s


If we check the shape of the produced array, we find that we have quite reduced the dimensionality of the problem but, are we still able to classify the different sleep stages with this features?

In [ ]:
bandpowers.shape

(718, 84)

## Simple model

> To check the usability of this features we will train a very simple model performing a random split within the same recording. As we are building our project sequentially, we want to make sure that the things we do are usable as building blocks for the next things.

First of all, we can extract the labels from the `epochs` object we have already created:

In [ ]:
labels = epochs.events[:,-1]
labels.shape

(718,)

Next, we can randomly split our data and train a simple default random forest:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(bandpowers, labels, test_size=0.2, random_state=42)

assert X_train.shape[0] == Y_train.shape[0]
assert X_test.shape[0] == Y_test.shape[0]
assert X_train.shape[1] == X_test.shape[1]

In [ ]:
%%time
model = RandomForestClassifier(random_state=42)
model.fit(X_train, Y_train)

CPU times: user 423 ms, sys: 3.37 ms, total: 426 ms
Wall time: 444 ms


RandomForestClassifier(random_state=42)

In [ ]:
model.score(X_train, Y_train), model.score(X_test, Y_test)

(1.0, 0.5972222222222222)

Having performed this simple check, we know that the features extracted hold, at least, some useful information that can be used to predict the different sleep stages. We can continue adding complexity to our pipeline.